In [ ]:
import re
import urllib

import benchlingapi
import pygsheets
import requests_html
import toml

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.workflow as workflow

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
col = workflow.get_strain_collection_sheets(gc.drive.service, "LIB")
col

In [ ]:
strain_sheet = gc.open_by_key(col["strains"]).worksheet()
plasmid_sheet = gc.open_by_key(col["plasmids"]).worksheet()

# Lab Obs ordering site

In [ ]:
import paulssonlab.api.labops as labops

In [ ]:
browser = await labops.login_harvard(
    config["ecommons"]["username"],
    config["ecommons"]["password"],
    service="https://sysbiolabops.hms.harvard.edu/casservice",
)

In [ ]:
addgene_orders = await labops.get_orders(browser, vendor="Addgene (26)")

In [ ]:
my_addgene = addgene_orders[
    addgene_orders["Name"].isin(["Jacob Shenker", "Noah Olsman"])
]

In [ ]:
addgene_catalog = (
    my_addgene["Catalog #"]
    .apply(lambda x: int(re.search(r"(\d+)$", x).group(1)))
    .values
)

# Densmore/3G/JUMP

In [ ]:
densmore_kit = "https://www.addgene.org/1000000059/"
threeg_kit = "https://www.addgene.org/1000000161/"
marionette_kit = "https://www.addgene.org/1000000137/"
jump_plasmids = [
    "https://www.addgene.org/126956/",
    "https://www.addgene.org/126959/",
    "https://www.addgene.org/126960/",
    "https://www.addgene.org/126961/",
    "https://www.addgene.org/126962/",
    "https://www.addgene.org/126963/",
    "https://www.addgene.org/126964/",
    "https://www.addgene.org/126965/",
    "https://www.addgene.org/126966/",
    "https://www.addgene.org/126967/",
    "https://www.addgene.org/126973/",
    "https://www.addgene.org/126974/",
    "https://www.addgene.org/126975/",
    "https://www.addgene.org/126976/",
    "https://www.addgene.org/126991/",
    "https://www.addgene.org/126996/",
    "https://www.addgene.org/127015/",
    "https://www.addgene.org/127047/",
    "https://www.addgene.org/127051/",
    "https://www.addgene.org/127025/",
    "https://www.addgene.org/127000/",
    "https://www.addgene.org/126983/",
]
jump_plasmids = sorted(jump_plasmids)

In [ ]:
addgene_plasmids = [
    f"https://www.addgene.org/{catalog}/"
    for catalog in addgene_catalog
    if catalog not in (1000000059,)  # remove Densmore
]

In [ ]:
all_addgene = [threeg_kit, *jump_plasmids, marionette_kit, *addgene_plasmids]

In [ ]:
# only include first for duplicate URLs
all_addgene_deduplicated = list(dict.fromkeys(all_addgene))

In [ ]:
len(all_addgene)

In [ ]:
len(all_addgene_deduplicated)

In [ ]:
addgene_data = workflow.import_addgene(
    all_addgene_deduplicated, strain_sheet, plasmid_sheet, col["plasmid_maps"]
)

# Test

In [ ]:
dd[0]["strain"]

In [ ]:
dd = workflow._import_addgene_data("https://www.addgene.org/1000000137/", callback=cb)

In [ ]:
z = [x["strain"] for x in dd if "plasmid_map" not in x]

In [ ]:
z

In [ ]:
[x["plasmid_map"].description for x in dd if "plasmid_map" in x]

In [ ]:
def cb(entry, data):
    print(data)
    return entry


d = workflow._import_addgene_data(
    "https://www.addgene.org/108512/", strain_overrides={"Tags": "foo"}, callback=cb
)

In [ ]:
d

In [ ]:
d[0]["strain"]["Aliases*"]

In [ ]:
d[0]["plasmid_map"].description.split(" ")[0]

In [ ]:
d2 = workflow.import_addgene(
    # "https://www.addgene.org/108512/",
    "https://www.addgene.org/1000000137/",
    strain_sheet,
    plasmid_sheet,
    col["plasmid_maps"],
    strain_overrides={"Tags": "foo"},
)